In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("../Files/AirPassengers.csv")
df.TravelDate = pd.to_datetime(df.TravelDate)

In [ ]:
df.Month = df.TravelDate

df = df.set_index('TravelDate')


In [ ]:
df.head()

In [ ]:
train, test = df[:-24], df[-24:]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test  = scaler.transform(test)

In [ ]:
n_input = test.shape[0]
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit_generator(generator, epochs=200)

In [ ]:
pred_list = []
batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)



In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=df[-n_input:].index, columns=['Predictions'])


In [ ]:
df_predict.head()

In [ ]:
df_test = pd.concat([df, df_predict], axis=1)

In [ ]:
df_test.tail(13)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df_test.index, df_test['Passengers'])
plt.plot(df_test.index, df_test['Predictions'], color='r')

In [ ]:
###Predictions for the next months

In [ ]:
train = df
scaler.fit(train)
train = scaler.transform(train)

n_input = test.shape[0]
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
model.fit_generator(generator, epochs=200)

pred_list = []
batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [ ]:
from pandas.tseries.offsets import DateOffset
add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,25)]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                         index=future_dates[-n_input:].index,
                         columns=['Prediction'])
df_proj = pd.concat([df, df_predict],axis=1)

In [ ]:
future_dates.tail(12)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df_proj.index, df_proj['Passengers'])
plt.plot(df_proj.index, df_proj['Prediction'], color='r')
plt.show()

In [ ]:
#end